### 1. Objetivos
**Geral**: encontrar RNA de plantas tidos como não codificantes mas que, na verdade, codificam proteínas

**Específico**: Varrer os bancos de dados de proteômica em busca de proteínas candidatas a serem codificadas por mRNAs tidos como não codificantes


### 2. PRoteomcis IDEntifications Database (PRIDE) [link](https://www.ebi.ac.uk/pride/)

Visão geral do PRIDE

In [87]:
from filter_datasets import *
import pandas as pd
import json
import collections
import seaborn as sn
% matplotlib inline
import matplotlib.pyplot as plt
import ppx #A Python interface to proteomics data repositories
from pyteomics import mztab
from ete3 import Tree, NodeStyle, TreeStyle, NodeStyle, faces, AttrFace, CircleFace
from ete3 import NCBITaxa
import glob
import urllib.parse
import urllib.request
import uniprot
import requests
from Bio import SeqIO
from io import StringIO
from pandas_profiling import ProfileReport

UsageError: Line magic function `%` not found.


In [5]:
# pride_plant.json foi gerado usando a função filterPride para checar o número de projetos com plantas 
with open('pride_plant.json') as pride_plants_file:
    pride_plant = json.load(pride_plants_file)
print(f"Total de projetos com plantas no PRIDE: {len(pride_plant)}")

Total de projetos com plantas no PRIDE: 1464


In [6]:
pd.DataFrame(pride_plant,).T.head()

,name,accession
PRD000037,Chlamydomonas reinhardtii,3055
PRD000044,Arabidopsis thaliana (mouse-ear cress),3702
PRD000051,Zea mays (maize),4577
PRD000084,Arabidopsis thaliana (mouse-ear cress),3702
PRD000096,Arabidopsis thaliana (mouse-ear cress),3702


In [7]:
# Pride_plants_mztab.json foi gerado usando a função get_mztabes. Ele contem os projetos com plantas e com arquivos mztab
with open("pride_plants_mztab.json") as mztabs:
    pride_plants_mztab = json.load(mztabs)
print(f"Total de projetos com arquivos mztab: {len(pride_plants_mztab)}")

Total de projetos com arquivos mztab: 270


Arquivos mztab por espécie

In [8]:
pride_plants_names = [values["name"] for keys,values in pride_plant.items()]
pride_plants_counter = collections.Counter(pride_plants_names)
plant_names = [plants[0] for plants in pride_plants_counter.items()]
plant_quant = [plants[1] for plants in pride_plants_counter.items()]
plant_with_mztab = [] 
for specie in pride_plants_counter.keys():
    all_projects = [projects for projects,values in pride_plant.items() if f"{specie}" == values["name"]]
    mztab_projects = list(set(pride_plants_mztab.keys()) & set(all_projects))
    plant_with_mztab.append(len(mztab_projects))
plants_count_dataframe = pd.DataFrame({
    "Specie":plant_names,
    "Total_projects":plant_quant,
    "Projects_with_mztab":plant_with_mztab
})
plants_count_dataframe.sort_values("Total_projects", ascending = False, inplace = True)


In [92]:
plt.figure(figsize = (12,8))
plot = plt.subplot()
projects = plot.barh(plants_count_dataframe.Specie[:20],plants_count_dataframe.Total_projects[:20],color = 'b',align='center', alpha = .6)
mztabs = plot.barh(plants_count_dataframe.Specie[:20],plants_count_dataframe.Projects_with_mztab[:20],color = 'g',align='center', alpha = 0.6,)
plot.set_ylabel("Espécie",fontsize = 20)
plot.set_xlabel("Número de projetos",fontsize = 20)
plot.set_title("20 plantas mais comuns (PRIDE)", fontsize = 30)
plot.tick_params(labelsize = 18)
plot.legend((projects,mztabs),("Total de projetos","Projetos com arquivos mztab"),fontsize = 20)
#plt.savefig("/home/tiago/documents/lncRNA/figures/most_commum.png")
plt.show()


### 3. Caminhos possíveis 

1. Os arquivos do PRIDE contem peptídeos que fazem parte de proteínas não caracterizadas e que podem ser mapeadas de volta em regiões codificam RNAs não codificantes nos genomas das espécias alvo.

2. Os arquivos do PRIDE contem peptídeos qua não estão nos proteoma das espécies alvo.


![Fluxograma](figures/fluxograma.jpg)

In [10]:
def get_projects(dict_plants = pride_plant, dict_mztab = pride_plants_mztab, specie = None):
    """
    Returns all projecst with mztab files for a give plant specie. Search by name
    parameters:
        dict_plants: contains all plant projects retrivied from PRIDE
        pride_plants_mztab: contains all plant projects with mztab retrivied from PRIDE
        specie: specie's name to search
    """
    projetcs_total = [projects for projects,values in dict_plants.items() if f"{specie}" in values["name"]]
    projects = list(set(dict_mztab.keys()) & set(projetcs_total))
    return projects

In [21]:
def download_projects(project_list):
    """
        Download all mztab files as well as creates a directory with all files available for download
        Parameters:
            project_list: list of projects to download
    """
    for projects in project_list:
        print(f"Downloading projetc:{projects}") 
        pride_obj = ppx.PrideProject(
        projects,
        local = os.path.join(os.getcwd(),mztab_storage_folder,f"{projects}")
        )
        pride_obj.download(pride_plants_mztab[projects])
        


In [62]:
def unzip_mztabs(path):
    """
    Unzip all mztba.gz files
    """
    for proj_dir,subdir,files in os.walk(os.path.join(os.getcwd(),mztab_storage_folder)):
        if proj_dir.endswith("/generated"):
           os.system(f"gunzip {os.path.join(proj_dir,'*.gz')}")


In [13]:
def get_allmztab(path):
    """
    Get a list with all mztab files 
    """
    all_files = glob.glob(path)
    return all_files

In [14]:
# usar accession de PSM para pegar description em PRT
def PSM2PRT (mztab,accession):
    """
    Uses mztab["PRT"] accession to get pep sequence description availabel only on mztab["PSM"]
    Parameters:
        mztab: file to search
        accession: accession number
    """   
    for pst_data in mztab["PRT"].itertuples():
            if pst_data.accession == accession:
                return pst_data.description


In [15]:
def resume_mztab(file_list:list,specie ):
    """
        Returns a fasta file, per project, with all peps on all mztabs
        Parameters:
            file_list: list containg mztab files
            specie: specie to name the returning fasta file   
    """
    for files in file_list:
        mztab_obj = mztab.MzTab(files)
        projetc_number  = mztab_obj.metadata["mzTab-ID"]
        with open(f"{specie}_{projetc_number}.fasta","w+") as file_fasta:
            for info in mztab_obj["PSM"].itertuples():
                file_fasta.write(f">{info.accession} | {PSM2PRT(mztab_obj,info.accession)} \n") # Se possível, preciso melhorar isso. O(n²) é lento
                file_fasta.write(f"{info.sequence}\n") 
    return file_fasta

In [16]:
def get_swissprot_sequences(accession:list,out_file_name:str,project):
    """
    convert pride accession number to swissprot accession number then downloads complete protein sequences
        Parameters:
            accession: List containg PRIDE IDs

    """
    databases_url = 'https://www.uniprot.org/uploadlists/'
    query = None
    if len(accession) == 1:
        query = accession
        params = {
        'from': 'ACC+ID', #pride accession
        'to': 'SWISSPROT', #swissprot accession
        'format': 'list',
        'query': f'{query}'}
    else:
        params = {
        'from': 'ACC+ID', #pride accession
        'to': 'SWISSPROT', #swissprot accession
        'format': 'list',
        'query': f'{" ".join(accession)}'}
    pride_accession = urllib.parse.urlencode(params)
    pride_accession = pride_accession.encode('utf-8')
    req = urllib.request.Request(databases_url, pride_accession)
    with urllib.request.urlopen(req) as accession_file:
        response = accession_file.read()
    swissprot_accession = list(response.decode('utf-8').split("\n"))

    
    # Get protein sequences from swissprot
    swissprot_url = "http://www.uniprot.org/uniprot/"
    with open(f"{'_'.join(out_file_name)}_{project}.complete.fasta","w+") as fasta:
        for IDs in swissprot_accession:
            joint_url = swissprot_url + IDs + ".fasta"
            swissprot_response = requests.post(joint_url)
            raw_data = "".join(swissprot_response.text)
            Seq = StringIO(raw_data)
            for seq_info in SeqIO.parse(Seq,'fasta'):
                fasta.write(f">{seq_info.description}\n")
                fasta.write(f"{seq_info.seq}\n")
    return fasta

In [17]:
def concatenate_swissprot(file_list, specie):
    """
    Concatenate get_swissprot_sequences results
    """
    for files in file_list:
        mztab_obj = mztab.MzTab(files)
        project_number  = mztab_obj.metadata["mzTab-ID"]
        accession_list = mztab_obj["PRT"].accession.values
        get_swissprot_sequences(accession_list,specie,project_number)

In [77]:
def download_sequences(mztab_prt,out_file):
    project_id  = mztab_prt.metadata["mzTab-ID"]
    with open(f"{os.path.join(out_file,project_id)}.fasta","w+") as sequence_fasta_file:
        for data in mztab_prt["PRT"].itertuples():
            uniprot_url = 'https://www.uniprot.org/uploadlists/'
            params = {
            'from': 'ACC+ID', #pride accession
            'to': 'EMBL', #ENA accession
            'format': 'list',
            'query': f'{data.accession}'
            }
            pride_accession = urllib.parse.urlencode(params)
            pride_accession = pride_accession.encode('utf-8')
            pride_request = urllib.request.Request(uniprot_url, pride_accession)
            with urllib.request.urlopen(pride_request) as request_data:
                request_response = request_data.read()
            ENA_accession = request_response.decode("utf-8").split('\n')
            ENA_ulr = url = f"https://www.ebi.ac.uk/ena/browser/api/fasta/{ENA_accession[0]}?lineLimit=1000"
            ENA_seq_request = requests.get(url).text.split("\n")
            fasta_id = data.description
            fasta_seq = "".join(ENA_seq_request[1:])
            sequence_fasta_file.write(f">{fasta_id}\n")
            sequence_fasta_file.write(f"{fasta_seq}\n")

## Resultados

### *Cafea arabica*

In [82]:
mztab_storage_folder = "C.arabica"
try:
    os.makedirs(os.path.join(os.getcwd(),mztab_storage_folder))
except:
    print(f"Folder already exist: {os.path.join(os.getcwd(),mztab_storage_folder)}")

Folder already exist: /home/tiago/documents/lncRNA/C.arabica


In [85]:
specie = "Coffea arabica"
projects = get_projects(specie = specie)
download_projects(projects)
unzip_mztabs(path = os.path.join(os.getcwd(),mztab_storage_folder))
for proj_dir,subdir,files in os.walk(os.path.join(os.getcwd(),mztab_storage_folder)):
    if proj_dir.endswith("/generated"):
        all_mztab_files = get_allmztab(os.path.join(proj_dir,"*.mztab"))
        for mztab_path in all_mztab_files:
            mztab_obj = mztab.MzTab(mztab_path)
            print(proj_dir.split("/")[-2])
            download_sequences(mztab_obj,proj_dir)

TOTAL:   0%|          | 0/1 [00:00<?, ?files/s]

generated/2experimentos.pride.mztab.gz:   0%|          | 0.00/846k [00:00<?, ?b/s]

gzip: /home/tiago/documents/lncRNA/C.arabica/PXD002963/generated/2experimentos.pride.mztab already exists;	not overwritten


PXD002963


### Soy bean

In [94]:
mztab_storage_folder = "G.max"
try:
    os.makedirs(os.path.join(os.getcwd(),mztab_storage_folder))
except:
    print(f"Folder already exist: {os.path.join(os.getcwd(),mztab_storage_folder)}")

Folder already exist: /home/tiago/documents/lncRNA/G.max


In [96]:
specie = "Glycine max"
projects = get_projects(specie = specie)
print(f"Total of projects for {specie}:{len(projects)}")
download_projects(projects)
unzip_mztabs(path = os.path.join(os.getcwd(),mztab_storage_folder))
for proj_dir,subdir,files in os.walk(os.path.join(os.getcwd(),mztab_storage_folder)):
    if proj_dir.endswith("/generated"):
        all_mztab_files = get_allmztab(os.path.join(proj_dir,"*.mztab"))
        for mztab_path in all_mztab_files:
            mztab_obj = mztab.MzTab(mztab_path)
            print(proj_dir.split("/")[-2])
            download_sequences(mztab_obj,proj_dir) 

Total of projects for Glycine max:7


TOTAL:   0%|          | 0/137 [00:00<?, ?files/s]

generated/SoySEC_standards_02a_03132018.MSGF+.pride.mztab.gz:   0%|          | 0.00/14.5M [00:00<?, ?b/s]

generated/SoySEC_standards_03a_03142018.MSGF+.pride.mztab.gz:   0%|          | 0.00/16.3M [00:00<?, ?b/s]

generated/SoySEC_standards_04a_03142018.MSGF+.pride.mztab.gz:   0%|          | 0.00/16.7M [00:00<?, ?b/s]

...

generated/B1.pride.mztab.gz:   0%|          | 0.00/74.6k [00:00<?, ?b/s]

generated/B2.pride.mztab.gz:   0%|          | 0.00/59.7k [00:00<?, ?b/s]

generated/B3.pride.mztab.gz:   0%|          | 0.00/64.2k [00:00<?, ?b/s]

gzip: /home/tiago/documents/lncRNA/G.max/PXD000160/generated/PRIDE_Exp_Complete_Ac_28646.pride.mztab already exists;	not overwritten
gzip: /home/tiago/documents/lncRNA/G.max/PXD000160/generated/PRIDE_Exp_Complete_Ac_28647.pride.mztab already exists;	not overwritten
gzip: /home/tiago/documents/lncRNA/G.max/PXD000160/generated/PRIDE_Exp_Complete_Ac_28648.pride.mztab already exists;	not overwritten
gzip: /home/tiago/documents/lncRNA/G.max/PXD000160/generated/PRIDE_Exp_Complete_Ac_28649.pride.mztab already exists;	not overwritten
gzip: /home/tiago/documents/lncRNA/G.max/PXD000160/generated/PRIDE_Exp_Complete_Ac_28650.pride.mztab already exists;	not overwritten
gzip: /home/tiago/documents/lncRNA/G.max/PXD000160/generated/PRIDE_Exp_Complete_Ac_28651.pride.mztab already exists;	not overwritten
gzip: /home/tiago/documents/lncRNA/G.max/PXD000160/generated/PRIDE_Exp_Complete_Ac_28652.pride.mztab already exists;	not overwritten
gzip: /home/tiago/documents/lncRNA/G.max/PXD000160/generated/PRIDE_Ex

PXD000160


### *Arabidopsis thaliana*

In [23]:
mztab_storage_folder = "A.thaliana"
try:
    os.makedirs(os.path.join(os.getcwd(),mztab_storage_folder))
except:
    print(f"Folder already exist: {os.path.join(os.getcwd(),mztab_storage_folder)}")

In [24]:
specie = "Arabidopsis thaliana"
projects = get_projects(specie = specie)
download_projects(projects)
unzip_mztabs(path = os.path.join(os.getcwd(),mztab_storage_folder))
for proj_dir,subdir,files in os.walk(os.path.join(os.getcwd(),mztab_storage_folder)):
    if proj_dir.endswith("/generated"):
        all_mztab_files = get_allmztab(os.path.join(proj_dir,"*.mztab"))
        for mztab_path in all_mztab_files:
            mztab_obj = mztab.MzTab(mztab_path)
            print(proj_dir.split("/")[-2])
            download_sequences(mztab_obj,proj_dir)

110
['PXD006328', 'PXD002606', 'PRD000634', 'PXD000136', 'PXD000692', 'PXD004596', 'PRD000156', 'PXD001344', 'PXD004267', 'PRD000392', 'PXD001880', 'PXD000908', 'PXD005179', 'PXD002476', 'PXD003103', 'PXD005740', 'PRD000635', 'PXD000869', 'PXD000880', 'PXD001286', 'PXD001878', 'PXD000280', 'PXD001885', 'PXD006807', 'PXD013321', 'PXD000235', 'PXD006880', 'PXD001134', 'PRD000737', 'PXD001877', 'PXD010518', 'PXD002186', 'PXD000211', 'PXD007979', 'PRD000314', 'PXD003352', 'PXD002396', 'PXD010104', 'PXD000567', 'PXD002075', 'PXD000566', 'PXD001386', 'PXD001879', 'PXD005168', 'PXD001884', 'PXD013999', 'PXD001252', 'PXD004027', 'PXD009392', 'PXD004763', 'PRD000246', 'PXD006848', 'PRD000581', 'PXD009510', 'PXD001883', 'PRD000182', 'PXD000500', 'PXD001400', 'PXD000223', 'PXD010816', 'PXD001562', 'PXD004896', 'PXD000417', 'PRD000096', 'PRD000508', 'PXD001795', 'PXD004025', 'PXD001349', 'PRD000289', 'PXD000942', 'PRD000531', 'PXD006598', 'PXD007678', 'PRD000717', 'PXD001179', 'PXD013264', 'PXD003